# Pre-submission metrics

This script extracts the metrics on the pre-submission set for teams to get an estimate of how their models are performing in the test set.

It is not relevant after the 2020 MIDL workshop, since future submissions will be reported only in the full test set.

In [ ]:
%matplotlib inline
import sys
import numpy as np
import os
import glob
import pickle
import h5py
from mrrec.metrics import metrics


results_path = "../../../Results-submissions/"

team_name = "ResoNNance/submission-03-August-2020"

reference = "reference"

nfiles = 4
nslices = 156

res_dic = {}

files12 = [ "e15790s3_P01536.7.h5","e14939s3_P44032.7.h5", "e13991s3_P01536.7.h5", "e15046s3_P59392.7.h5", \
          "e15791s4_P11264.7.h5"]# 
        
files32 = ["e16578s3_P19968.7.h5", "e16842s3_P55296.7.h5", "e16214s13_P25088.7.h5", "e16587s3_P20992.7.h5",\
           "e16869s3_P03584.7.h5"]

recs12 = [os.path.join(results_path,team_name,"Track01","12-channel-R=5"),\
        os.path.join(results_path,team_name,"Track01","12-channel-R=10"),\
        os.path.join(results_path,team_name,"Track02","12-channel-R=5"),\
        os.path.join(results_path,team_name,"Track02","12-channel-R=10")]

recs32 = [os.path.join(results_path,team_name,"Track02","32-channel-R=5"),\
        os.path.join(results_path,team_name,"Track02","32-channel-R=10")]

refs12 = [os.path.join(results_path,reference,"12-channel")]*4

refs32 = [os.path.join(results_path,reference,"32-channel")]*2


dic_keys12 = ["track01_12_channel_R=5","track01_12_channel_R=10","track02_12_channel_R=5","track02_12_channel_R=10"]
dic_keys32 = ["track02_32_channel_R=5","track02_32_channel_R=10"]

In [2]:
for (i,rec_path) in enumerate(recs12):
    nfiles = len(files12)
    m = np.zeros((nfiles*nslices,3))
    
    for (jj,name) in enumerate(files12):
        print(name)
        try:
            with h5py.File(os.path.join(rec_path,name), 'r') as f:
                rec = f['reconstruction'][()]
        except:
            continue
        with h5py.File(os.path.join(refs12[i],name), 'r') as f:
            ref = f['reconstruction'][()]
        
        ref_max = ref.max(axis = (1,2),keepdims = True)
        ref = ref/ref_max
        rec = rec/ref_max
        
        
        ssim,psnr,vif = metrics(rec,ref)    
        m[jj*nslices:(jj+1)*nslices,0] = ssim
        m[jj*nslices:(jj+1)*nslices,1] = psnr
        m[jj*nslices:(jj+1)*nslices,2] = vif

    res_dic[dic_keys12[i]] = m    


for (i,rec_path) in enumerate(recs32):
    nfiles = len(files32)
    m = np.zeros((nfiles*nslices,3))
    
    for (jj,name) in enumerate(files32):
        print(name)
        try:
            with h5py.File(os.path.join(rec_path,name), 'r') as f:
                rec = f['reconstruction'][()]
        except:
            continue
        with h5py.File(os.path.join(refs32[i],name), 'r') as f:
            ref = f['reconstruction'][()]
        
        ref_max = ref.max(axis = (1,2),keepdims = True)
        ref = ref/ref_max
        rec = rec/ref_max
        
        ssim,psnr,vif = metrics(rec,ref)    
        m[jj*nslices:(jj+1)*nslices,0] = ssim
        m[jj*nslices:(jj+1)*nslices,1] = psnr
        m[jj*nslices:(jj+1)*nslices,2] = vif

    res_dic[dic_keys32[i]] = m                         

e15790s3_P01536.7.h5
e14939s3_P44032.7.h5
e13991s3_P01536.7.h5
e15046s3_P59392.7.h5
e15791s4_P11264.7.h5
e15790s3_P01536.7.h5
e14939s3_P44032.7.h5
e13991s3_P01536.7.h5
e15046s3_P59392.7.h5
e15791s4_P11264.7.h5
e15790s3_P01536.7.h5
e14939s3_P44032.7.h5
e13991s3_P01536.7.h5
e15046s3_P59392.7.h5
e15791s4_P11264.7.h5
e15790s3_P01536.7.h5
e14939s3_P44032.7.h5
e13991s3_P01536.7.h5
e15046s3_P59392.7.h5
e15791s4_P11264.7.h5
e16578s3_P19968.7.h5
e16842s3_P55296.7.h5
e16214s13_P25088.7.h5
e16587s3_P20992.7.h5
e16869s3_P03584.7.h5
e16578s3_P19968.7.h5
e16842s3_P55296.7.h5
e16214s13_P25088.7.h5
e16587s3_P20992.7.h5
e16869s3_P03584.7.h5


In [3]:
out_file = os.path.join(results_path,'Metrics-pre-submission',team_name.replace("/","-",1) + '.pickle')
with open(out_file, 'wb') as handle:
    pickle.dump(res_dic, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
print("Track 01 - 12-channel")

t1_r5 = 'track01_12_channel_R=5'
name = team_name.replace("/submission-"," - ",1)

res_file = open(out_file,'rb')
results = pickle.load(res_file)
res_file.close()

if t1_r5 in results:
    res = results[t1_r5]
    ssim = res[:,0].reshape(nslices,nfiles)
    psnr = res[:,1].reshape(nslices,nfiles)
    vif = res[:,2].reshape(nslices,nfiles)
    aux = (ssim.mean(),ssim.std(),psnr.mean(),psnr.std(),vif.mean(),vif.std())

    print("$R = 5 $ & $%.4f \\pm %.4f$ &  $%.2f \\pm %.2f$ &  $%.4f \\pm %.4f$" %aux)


t1_r10 = 'track01_12_channel_R=10'
    
if t1_r10 in results:
    res = results[t1_r10]
    ssim = res[:,0].reshape(nslices,nfiles)
    psnr = res[:,1].reshape(nslices,nfiles)
    vif = res[:,2].reshape(nslices,nfiles)
    aux = (ssim.mean(),ssim.std(),psnr.mean(),psnr.std(),vif.mean(),vif.std())

    print("$R = 10$ & $%.4f \\pm %.4f$ &  $%.2f \\pm %.2f$ &  $%.4f \\pm %.4f$" %aux)

Track 01 - 12-channel
$R = 5 $ & $0.9339 \pm 0.0137$ &  $34.64 \pm 1.80$ &  $0.9589 \pm 0.0132$
$R = 10$ & $0.7677 \pm 0.0374$ &  $26.19 \pm 1.45$ &  $0.7159 \pm 0.0573$


In [13]:
print("Track 02 - 12-channel")

t2_c12_r5 = 'track02_12_channel_R=5'
if t2_c12_r5 in results:
    res = results[t2_c12_r5]
    ssim = res[:,0].reshape(nslices,nfiles)
    psnr = res[:,1].reshape(nslices,nfiles)
    vif = res[:,2].reshape(nslices,nfiles)
    aux = (ssim.mean(),ssim.std(),psnr.mean(),psnr.std(),vif.mean(),vif.std())

    print("$R = 5 $ & $%.4f \\pm %.4f$ &  $%.2f \\pm %.2f$ &  $%.4f \\pm %.4f$" %aux)


t2_c12_r10 = 'track02_12_channel_R=10'

if t2_c12_r10 in results:
    res = results[t2_c12_r10]
    ssim = res[:,0].reshape(nslices,nfiles)
    psnr = res[:,1].reshape(nslices,nfiles)
    vif = res[:,2].reshape(nslices,nfiles)
    aux = (ssim.mean(),ssim.std(),psnr.mean(),psnr.std(),vif.mean(),vif.std())

    print("$R = 10$ & $%.4f \\pm %.4f$ &  $%.2f \\pm %.2f$ &  $%.4f \\pm %.4f$" %aux)

Track 02 - 12-channel
$R = 5 $ & $0.9339 \pm 0.0137$ &  $34.64 \pm 1.80$ &  $0.9589 \pm 0.0132$
$R = 10$ & $0.7677 \pm 0.0374$ &  $26.19 \pm 1.45$ &  $0.7159 \pm 0.0573$


In [15]:
t2_c32_r5 = 'track02_32_channel_R=5'


if t2_c32_r5 in results:
    res = results[t2_c32_r5]
    ssim = res[:,0].reshape(nslices,nfiles)
    psnr = res[:,1].reshape(nslices,nfiles)
    vif = res[:,2].reshape(nslices,nfiles)
    aux = (ssim.mean(),ssim.std(),psnr.mean(),psnr.std(),vif.mean(),vif.std())

    print("$R = 5 $ & $%.4f \\pm %.4f$ &  $%.2f \\pm %.2f$ &  $%.4f \\pm %.4f$" %aux)

t2_c32_r10 = 'track02_32_channel_R=10'

aux = []    
if t2_c32_r10 in results:
    res = results[t2_c32_r10]
    ssim = res[:,0].reshape(nslices,nfiles)
    psnr = res[:,1].reshape(nslices,nfiles)
    vif = res[:,2].reshape(nslices,nfiles)
    aux = (ssim.mean(),ssim.std(),psnr.mean(),psnr.std(),vif.mean(),vif.std())
    print("$R = 10$ & $%.4f \\pm %.4f$ &  $%.2f \\pm %.2f$ &  $%.4f \\pm %.4f$" %aux)

$R = 5 $ & $0.9439 \pm 0.0199$ &  $35.23 \pm 2.56$ &  $0.9920 \pm 0.0617$
$R = 10$ & $0.9055 \pm 0.0302$ &  $32.81 \pm 2.53$ &  $0.9637 \pm 0.0691$
